In [1]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## For the sake of brevity...
T, F = True, False

In [75]:
def P(var, value, evidence={}):
    '''The probability distribution for P(var | evidence), 
    when all parent variables are known (in evidence)'''
    if len(var.parents)==1:
        # only one parent
        print(var.variable)
        print(var.parents[0])
        print(evidence)
        row = evidence[var.parents[0]]
    else:
        # multiple parents
        row = tuple(evidence[parent] for parent in var.parents)
    return var.cpt[row] if value else 1-var.cpt[row]

In [3]:

class BayesNode:
    
    def __init__(self, name, parents, values, cpt):
        if isinstance(parents, str):
            parents = parents.split()
            
        if len(parents)==0:
            # if no parents, empty dict key for cpt
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # if there is only one parent, only one tuple argument
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v): p for v, p in cpt.items()}

        self.variable = name
        self.parents = parents
        self.cpt = cpt
        self.values = values
        self.children = []
        
    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))    

In [8]:

    
##===============================================##
## Suggested skeleton codes for a BayesNet class ##
##===============================================##

class BayesNet:
    '''Bayesian network containing only boolean-variable nodes.'''

    def __init__(self, nodes):
        '''Initialize the Bayes net by adding each of the nodes,
        which should be a list BayesNode class objects ordered
        from parents to children (`top` to `bottom`, from causes
        to effects)'''
        
        # your code goes here...
        self.nodes = nodes
        

                
    def add(self, node):
        '''Add a new BayesNode to the BayesNet. The parents should all
        already be in the net, and the variable itself should not be'''
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        
#         for i in range(self.nodes):
#             if len(node.parents) >= len(self.nodes[i].parents):
#                 self.nodes = self.nodes.insert(node, i)
        self.nodes.append(node)
        

            
    def find_node(self, var):
        '''Find and return the BayesNode in the net with name `var`'''
        
        for node in self.nodes:
            if node.variable == var:
                return node
        

        
    def find_values(self, var):
        '''Return the set of possible values for variable `var`'''
        
        return self.find_node(var).values
        

    
    def __repr__(self):
        return 'BayesNet({})'.format(self.nodes)

In [5]:
class Tests_Problem1(unittest.TestCase):
    def setUp(self):
        self.p1 = BayesNode('p1', '', [T,F], 0.3)
        self.p2 = BayesNode('p2', '', [T,F], 0.6)
        self.c  = BayesNode('c', ['p1', 'p2'], [T,F], {(T,T):0.1, (T,F):0.2, (F,T):0.3, (F,F):0.4})
    def test_onenode(self):
        self.assertEqual(P(self.p1, T), 0.3)
    def test_twonode(self):
        self.assertEqual(P(self.c, F, {'p1':T, 'p2':F}), 0.8)

In [6]:
p1 = BayesNode('p1', '', [T,F], 0.3)
p2 = BayesNode('p2', '', [T,F], 0.6)
c  = BayesNode('c', ['p1', 'p2'], [T,F], {(T,T):0.1, (T,F):0.2, (F,T):0.3, (F,F):0.4})
#print probability value of p1 when it is true using the function 'P'
print("P1 = True:",P(p1, T))
#print probability value of 'C', given that 'p1' is True and 'p2' is False using the function 'P'
print("C=True:",P(c, F, {p1.variable:T, p2.variable:F}))

P1 = True: 0.3
C=True: 0.8


In [62]:
def get_prob(X, e, bn):
    if isinstance(X, str):
        node = bn.find_node(X)
    else:
        node = X
        
    probs = []
    e[node.variable] = T
    probs[0] = unknown_nodes(bn.nodes, e)
    e[node.variable] = F
    probs[1] = unknown_nodes(bn.nodes, e)
    
    probs.map(_/probs.sum())
    return probs

In [72]:
def unknown_nodes(nodes, e):
    if len(nodes) == 0:
        return 1.0
    var = nodes.pop()
    if var.variable in e:
        val = P(var, e[var.variable], e) * unknown_nodes(nodes, e)
        nodes.append(var)
        return val
    else:
        e[var.variable] = T
        print(e)
        total = P(var, T, e) * unknown_nodes(nodes, e)
        e[var.variable] = F
        total += P(var, F, e) * unknown_nodes(nodes, e)
        del e[var.variable]
        nodes.append(var)
        return total

In [73]:
Sm = BayesNode('Sm', [], [T, F], 0.2)
ME = BayesNode('ME', [], [T, F], 0.5)

HBP = BayesNode('HBP', ['Sm', 'ME'], [T, F], {(T, T): 0.6, (T, F): 0.72, (F, T): 0.33, (F, F): 0.51})
Ath = BayesNode('Ath', [], [T, F], 0.53)
FH = BayesNode('FH', [], [T, F], 0.15)

HD = BayesNode('HD', ['HBP', 'Ath', 'FH'], [T, F], {(T, T, T): 0.92, (T, T, F): 0.91, (T, F, T): 0.81, (T, F, F): 0.77, (F, T, T): 0.75, (F, T, F): 0.69, (F, F, T): 0.38, (F, F, F): 0.23})

Ang = BayesNode('Ang', ['HD'], [T, F], {T: 0.85, F: 0.4})
Rapid = BayesNode('Rapid', ['HD'], [T, F], {T: 0.99, F: 0.30})

BN = BayesNet([Sm, ME, HBP, Ath, FH, HD, Ang, Rapid])

In [78]:

a = get_prob(FH,{},BN)
print("1.  {:0.2f}".format(round(a[0],2)))

b= get_prob(Ang,{"HD" : T}, BN)
print("2.  {:0.2f}".format(round(b[1],2)))
# print("2. ",b[1])

c= get_prob(HBP,{"Sm" : T, "ME" : F}, BN)
print("3.  {:0.2f}".format(round(c[0],2)))
# print("3. ",c[0])

Ang
HD
{'FH': True, 'Sm': True, 'ME': True, 'HBP': True, 'Ath': True, 'HD': True, 'Ang': True}
Rapid
HD
{'FH': True, 'Sm': True, 'ME': True, 'HBP': True, 'Ath': True, 'HD': True, 'Ang': True, 'Rapid': True}
Rapid
HD
{'FH': True, 'Sm': True, 'ME': True, 'HBP': True, 'Ath': True, 'HD': True, 'Ang': True, 'Rapid': False}
Ang
HD
{'FH': True, 'Sm': True, 'ME': True, 'HBP': True, 'Ath': True, 'HD': True, 'Ang': False}
Rapid
HD
{'FH': True, 'Sm': True, 'ME': True, 'HBP': True, 'Ath': True, 'HD': True, 'Ang': False, 'Rapid': True}
Rapid
HD
{'FH': True, 'Sm': True, 'ME': True, 'HBP': True, 'Ath': True, 'HD': True, 'Ang': False, 'Rapid': False}
Ang
HD
{'FH': True, 'Sm': True, 'ME': True, 'HBP': True, 'Ath': True, 'HD': False, 'Ang': True}
Rapid
HD
{'FH': True, 'Sm': True, 'ME': True, 'HBP': True, 'Ath': True, 'HD': False, 'Ang': True, 'Rapid': True}
Rapid
HD
{'FH': True, 'Sm': True, 'ME': True, 'HBP': True, 'Ath': True, 'HD': False, 'Ang': True, 'Rapid': False}
Ang
HD
{'FH': True, 'Sm': True, 'M

In [55]:

d = get_prob(HD,{},BN)
# print(d[0])
print("{:0.2f}".format(round(d[0],2)))

d = get_prob(HD,{"Rapid" : T},BN)
# print(d[1])
print("{:0.2f}".format(round(d[1],2)))

d = get_prob(HBP,{"HD":T,"FH":T},BN)
# print(d[0])
print("{:0.2f}".format(round(d[0],2)))

d = get_prob(Sm,{"HD":T},BN)
# print(d[0])
print("{:0.2f}".format(round(d[0],2)))

d = get_prob(Sm,{"HD":T, "HBP":T},BN)
# print(d[0])
print("{:0.2f}".format(round(d[0],2)))

d = get_prob(Sm,{"HD":T, "HBP":T, "ME":F},BN)
# print(d[0])
print("{:0.2f}".format(round(d[0],2)))

0.66
0.14
0.57
0.22
0.28
0.26


In [77]:
varss = ['Rapid', 'Ang', 'HD', 'FH', 'Ath', 'HBP', 'ME', 'Sm']
# Please round your answer to two decimal places
# print("Answer =  {:0.01f}".format(answer))
# or
# round(answer,1)
#

sToObj = {
    'Sm': Sm,
    'ME': ME,
    'HBP': HBP,
    'Ath': Ath,
    'FH': FH,
    'HD': HD,
    'Ang': Ang,
    'Rapid': Rapid,
    'BN': BN
}

objToS = {
    Sm: 'Sm',
    ME: 'ME',
    HBP: 'HBP',
    Ath: 'Ath',
    FH: 'FH',
    HD: 'HD',
    Ang: 'Ang',
    Rapid: 'Rapid',
    BN: 'BN'
}

def enumAll(varss, e):
    if len(varss) == 0:
        return 1.0
    Y = varss.pop()

    objY = sToObj[Y]

    if Y in e:
        val = P(objY, e[Y], e) * enumAll(varss, e)
        varss.append(Y)
        return val
    else:
        total = 0
        e[Y] = T
        total += P(objY, T, e) * enumAll(varss, e)
        e[Y] = F
        total += P(objY, F, e) * enumAll(varss, e)
        del e[Y]
        varss.append(Y)
        return total


def get_prob(x, e, bn):
    # your code goes here...
    q = [0, 0]

    xs = objToS[x]

    e[xs] = T
    q[0] = enumAll(varss, e)
    del e[xs]

    e[xs] = F
    q[1] = enumAll(varss, e)
    del e[xs]

    summy = q[0] + q[1]
    q[0] /= summy
    q[1] /= summy

    return q